In [ ]:
!pip install google-genai pydantic

In [2]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [3]:
import os
from google import genai
from pydantic import BaseModel, Field

# Initialize the Google GenAI client (uses GEMINI_API_KEY from environment)
client = genai.Client()

# Set the model for our agents
MODEL_ID = 'gemini-2.5-flash'

# --- 1. Define the Structured Schema for the Critic ---
class CritiqueResult(BaseModel):
    quality_score: int = Field(description="Score from 1 to 10 based on code safety and quality.")
    meets_requirements: bool = Field(description="True if the code passes all security audits, False otherwise.")
    feedback: str = Field(description="Specific feedback on what needs to be fixed if requirements are not met.")

# --- 2. Define the Specialized Agents ---

def generator_agent(user_request: str, previous_feedback: str = "") -> str:
    """The Generator Agent creates or refines the initial output."""
    print("-> [Generator] Writing/Refining code...")

    if previous_feedback:
        prompt = f"""Revise the following code based on the security auditor's feedback.
        Original Request: {user_request}
        Critic Feedback: {previous_feedback}

        Return ONLY the raw Python code without markdown blocks."""
    else:
        prompt = f"""Write a Python function to fulfill this request: {user_request}

        Return ONLY the raw Python code without markdown blocks."""

    response = client.models.generate_content(model=MODEL_ID, contents=prompt)
    return response.text

def critic_agent(code: str) -> CritiqueResult:
    """The Critic Subagent acts as a security auditor checking against constraints."""
    print("-> [Critic Subagent] Auditing code for vulnerabilities...")

    prompt = f"""Act as a strict security auditor. Review the following Python code for potential vulnerabilities
    (like SQL injection, hardcoded credentials, or unsafe module usage).

    Code to review:
    {code}

    If it is perfectly safe and uses best practices, set meets_requirements to true and give a high quality_score.
    If there are any risks, set meets_requirements to false, give a lower quality_score, and provide specific feedback for revision."""

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': CritiqueResult,
            'temperature': 0.1, # Keep temperature low for deterministic auditing
        },
    )
    return response.parsed

# --- 3. Define the Orchestrator (Loop) ---

def review_and_critique_workflow(user_prompt: str, max_iterations: int = 3) -> str:
    """
    The orchestrator manages the generator-critic loop.
    It returns the response to the user only if the quality score meets requirements.
    """
    print(f"Starting Review & Critique Workflow for: '{user_prompt}'\n")

    iteration = 1
    feedback_history = ""
    current_code = ""

    while iteration <= max_iterations:
        print(f"--- Iteration {iteration} ---")

        # 1. Generator creates the initial output or refines based on feedback
        current_code = generator_agent(user_prompt, feedback_history)
        print(f"Generated Code:\n{current_code}\n")

        # 2. Send response for evaluation to the Critic
        evaluation = critic_agent(current_code)

        print(f"-> Quality Score: {evaluation.quality_score}/10")

        # 3. Check if quality score meets requirements
        if evaluation.meets_requirements:
            print("-> ✅ Code approved by Critic. Sending response to user.\n")
            return current_code
        else:
            print(f"-> ❌ Code rejected. Sending quality score and feedback back to Generator: {evaluation.feedback}\n")
            # Send quality score and feedback back to the Generator for the next loop
            feedback_history = evaluation.feedback
            iteration += 1

    print("-> ⚠️ Max iterations reached. The workflow accumulated too much latency/cost. Exiting.\n")
    return current_code

# --- Execution ---
if __name__ == "__main__":
    # Example request that might prompt unsafe code generation
    task = "Write a function that connects to a sqlite database and executes a user-provided SQL query string directly."

    # Trigger the workflow
    final_output = review_and_critique_workflow(task)

    print("=== Final Verified Output ===")
    print(final_output)

Starting Review & Critique Workflow for: 'Write a function that connects to a sqlite database and executes a user-provided SQL query string directly.'

--- Iteration 1 ---
-> [Generator] Writing/Refining code...
Generated Code:
import sqlite3

def execute_sqlite_query(db_path: str, sql_query: str):
    """
    Connects to a SQLite database and executes a user-provided SQL query string directly.

    Args:
        db_path (str): The path to the SQLite database file (e.g., 'my_database.db').
        sql_query (str): The SQL query string to be executed.

    Returns:
        list of tuples: If the query is a SELECT statement, returns the fetched rows as a list of tuples.
        None: If the query is a non-SELECT statement (e.g., INSERT, UPDATE, DELETE, DDL).
        Raises an exception if an error occurs during execution.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql_query)

        if sql_query.strip().